In [13]:
from datasets import load_dataset, Audio
from transformers import EncodecModel, AutoProcessor
import scipy
import numpy as np

# dummy dataset, however you can swap this with an dataset on the 🤗 hub or bring your own
librispeech_dummy = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")

# load the model + processor (for pre-processing the audio)
model = EncodecModel.from_pretrained("facebook/encodec_24khz")
processor = AutoProcessor.from_pretrained("facebook/encodec_24khz")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [16]:
#Write the original audio to a wav file for listening
scipy.io.wavfile.write("original.wav", librispeech_dummy[0]['audio']['sampling_rate'], librispeech_dummy[0]['audio']['array'])

In [18]:
# cast the audio data to the correct sampling rate for the model
librispeech_dummy = librispeech_dummy.cast_column("audio", Audio(sampling_rate=processor.sampling_rate))
audio_sample = librispeech_dummy[0]["audio"]["array"]

# pre-process the inputs
inputs = processor(raw_audio=audio_sample, sampling_rate=processor.sampling_rate, return_tensors="pt")

# explicitly encode then decode the audio inputs
encoder_outputs = model.encode(inputs["input_values"], inputs["padding_mask"])
audio_values = model.decode(encoder_outputs.audio_codes, encoder_outputs.audio_scales, inputs["padding_mask"])[0]

In [22]:
#Write the reconstructed audio to a wav file for listening
scipy.io.wavfile.write("reconstructed.wav", librispeech_dummy[0]['audio']['sampling_rate'], audio_values.detach().numpy())